In [ ]:
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
import pickle as pkl
import spacy
import copy


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df.head(10)

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
5,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...,16,talk.politics.guns
6,From: bmdelane@quads.uchicago.edu (brian manni...,13,sci.med
7,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...,3,comp.sys.ibm.pc.hardware
8,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...,2,comp.os.ms-windows.misc
9,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...,4,comp.sys.mac.hardware


In [ ]:
print(df.target_names.value_counts())
print(df.iloc[0,0])
# Target does match , Business question regarding topics people talk about which are not related to work

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
sci.med                     594
rec.autos                   594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: target_names, dtype: int64
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70

In [ ]:
def remove_line(df_row):
  df_row = re.sub('[Ff]rom:(.*?)\n', "", df_row)
  df_row =re.sub('[Ss]ubject:(.*?)\n', "", df_row)
  df_row =re.sub('[Aa]rticle-[Ii].[Dd].:(.*?)\n', "", df_row)
  df_row =re.sub('[Oo]rganization:(.*?)\n', "", df_row)
  df_row =re.sub('[Ll]ines:(.*?)\n', "", df_row)
  df_row =re.sub('[Nn][Nn][Tt][Pp]-[Pp]osting-[Hh]ost:(.*?)\n', "", df_row)
  df_row =re.sub('[Dd]istribution:(.*?)\n', "", df_row)
  df_row =re.sub('[Rr]eply-[Tt]o:(.*?)\n', "", df_row)
  df_row =re.sub('[Xx]-[Nn]ewsreader:(.*?)\n', "", df_row)
  df_row =re.sub('[Ee]xpires:(.*?)\n', "", df_row)
  df_row =re.sub('-(.*?)\n', "", df_row)
  df_row =re.sub('[Ss]ummary:', "", df_row)
  df_row =re.sub('[Kk]eywords:', "", df_row)
  return df_row


In [ ]:
s = df.iloc[1,0]
s = remove_line(s)
s

" Final call for SI clock reports\n SI,acceleration,clock,upgrade\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n"

In [ ]:
data = df["content"].apply(remove_line)

In [ ]:
data

0        \n I was wondering if anyone out there could e...
1         Final call for SI clock reports\n SI,accelera...
2        \nwell folks, my mac plus finally gave up the ...
3        \nRobert J.C. Kyanko (rob@rjck.UUCP) wrote:\n>...
4        \nFrom article <C5owCB.n3p@world.std.com>, by ...
                               ...                        
11309    \nDN> DN> A neurology\nDN> consultation is che...
11310    \nI have a (very old) Mac 512k and a Mac Plus,...
11311    \nI just installed a DX2cooler on the chip.  A...
11312    \nIn article <1qkgbuINNs9n@shelley.u.washingto...
11313     see above\n\nStolen from Pasadena between 4:3...
Name: content, Length: 11314, dtype: object

In [ ]:
data = data.apply(strip_numeric)
data = data.apply(strip_punctuation)
data = data.apply(strip_multiple_whitespaces)
data = data.str.lower()

In [ ]:
data

0         i was wondering if anyone out there could enl...
1         final call for si clock reports si accelerati...
2         well folks my mac plus finally gave up the gh...
3         robert j c kyanko rob rjck uucp wrote abraxis...
4         from article cowcb np world std com by tombak...
                               ...                        
11309     dn dn a neurology dn consultation is cheaper ...
11310     i have a very old mac k and a mac plus both o...
11311     i just installed a dxcooler on the chip after...
11312     in article qkgbuinnsn shelley u washington ed...
11313     see above stolen from pasadena between and pm...
Name: content, Length: 11314, dtype: object

In [ ]:
print(len(sorted(stopwords.words('english'))))

179


In [ ]:
len(STOPWORDS)

337

In [ ]:
data = data.apply(remove_stopwords)
print(data)
data = data.apply(strip_short)

0        wondering enlighten car saw day early s called...
1        final si clock reports si acceleration clock u...
2        folks mac plus finally gave ghost weekend star...
3        robert j c kyanko rob rjck uucp wrote abraxis ...
4        article cowcb np world std com tombaker world ...
                               ...                        
11309    dn dn neurology dn consultation cheaper scan b...
11310    old mac k mac plus problem screens blank minor...
11311    installed dxcooler chip hour weight cooler dis...
11312    article qkgbuinnsn shelley u washington edu bo...
11313    stolen pasadena pm blue white honda cbrrr cali...
Name: content, Length: 11314, dtype: object


In [ ]:
# apply stemming : in stemming non existing word occurs
data_copy = copy.deepcopy(data)
data_stemmed = data_copy.apply(stem_text)
data_stemmed


0        wonder enlighten car saw dai earli call brickl...
1        final clock report acceler clock upgrad fair n...
2        folk mac plu final gave ghost weekend start li...
3        robert kyanko rob rjck uucp wrote abraxi iasta...
4        articl cowcb world std com tombak world std co...
                               ...                        
11309    neurolog consult cheaper scan better neurologi...
11310    old mac mac plu problem screen blank minor phy...
11311    instal dxcooler chip hour weight cooler dislod...
11312    articl qkgbuinnsn shellei washington edu bolso...
11313    stolen pasadena blue white honda cbrrr califor...
Name: content, Length: 11314, dtype: object

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# store stemmed data 
filename = "/content/drive/MyDrive/stemmed_data.pkl"
pkl.dump(data_stemmed,open(filename, "wb"))

In [ ]:
# apply lemmatization 
nlp = en_core_web_sm.load()
data_lem = [nlp(t)for t in data]
data_lemma = [" ".join([token.lemma_ for token in t]) for t in data_lem]
data_lemma[0]


'wonder enlighten car see day early call bricklin door small addition bumper separate rest body know tellme model engine specs year production car history info funky look car thank'

In [ ]:
# stored lemma data
filename = "/content/drive/MyDrive/lemma_data.pkl"
pkl.dump(data_lemma,open(filename, "wb"))